In [1]:
import os
import sys
import pandas as pd
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline


#temporary fix for running pyspark in jupyter notebook
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
# initialize spark session
spark = SparkSession.builder.master("local[2]").appName("Victim-Records").getOrCreate()

sc = spark.sparkContext

sqlContext = SQLContext(sc)


c:\Users\kyleh\Documents\SENG550-Final-Project\.venv\Lib\site-packages\pyspark\sql\context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
# define the path to the data

VictimRecords = ['20160924_VictimRecords.txt','20170112_VictimRecords.txt','20180925_VictimRecords.txt', '20201024_VictimRecords.txt']
# define the schema
schema = StructType([
    StructField('CASE_ID', StringType(), True),
    StructField('PARTY_NUMBER', IntegerType(), True),
    StructField('VICTIM_ROLE', StringType(), True),
    StructField('VICTIM_SEX', StringType(), True),
    StructField('VICTIM_AGE', IntegerType(), True),
    StructField('VICTIM_DEGREE_OF_INJURY', StringType(), True),
    StructField('VICTIM_SEATING_POSITION', StringType(), True),
    StructField('VICTIM_SAFETY_EQUIP1', StringType(), True),
    StructField('VICTIM_SAFETY_EQUIP2', StringType(), True),
    StructField('VICTIM_EJECTED', StringType(), True),
])


# load the data, skip header
victim_df = spark.read.csv(path = VictimRecords, schema = schema).cache()
header = victim_df.first()
victim_df = victim_df.filter(col("CASE_ID") != header["CASE_ID"])
victim_df.take(5)




[Row(CASE_ID=' 097293', PARTY_NUMBER=1, VICTIM_ROLE='2', VICTIM_SEX='M', VICTIM_AGE=20, VICTIM_DEGREE_OF_INJURY='0', VICTIM_SEATING_POSITION='3', VICTIM_SAFETY_EQUIP1='G', VICTIM_SAFETY_EQUIP2='-', VICTIM_EJECTED='0'),
 Row(CASE_ID=' 965874', PARTY_NUMBER=2, VICTIM_ROLE='2', VICTIM_SEX='M', VICTIM_AGE=19, VICTIM_DEGREE_OF_INJURY='0', VICTIM_SEATING_POSITION='3', VICTIM_SAFETY_EQUIP1='M', VICTIM_SAFETY_EQUIP2='G', VICTIM_EJECTED='0'),
 Row(CASE_ID='0000003', PARTY_NUMBER=1, VICTIM_ROLE='2', VICTIM_SEX='M', VICTIM_AGE=21, VICTIM_DEGREE_OF_INJURY='0', VICTIM_SEATING_POSITION='3', VICTIM_SAFETY_EQUIP1='G', VICTIM_SAFETY_EQUIP2='-', VICTIM_EJECTED='0'),
 Row(CASE_ID='0000005', PARTY_NUMBER=1, VICTIM_ROLE='1', VICTIM_SEX='M', VICTIM_AGE=44, VICTIM_DEGREE_OF_INJURY='2', VICTIM_SEATING_POSITION='1', VICTIM_SAFETY_EQUIP1='G', VICTIM_SAFETY_EQUIP2='-', VICTIM_EJECTED='0'),
 Row(CASE_ID='0000008', PARTY_NUMBER=1, VICTIM_ROLE='2', VICTIM_SEX='F', VICTIM_AGE=59, VICTIM_DEGREE_OF_INJURY='0', VICTIM_

In [4]:
victim_df.columns


['CASE_ID',
 'PARTY_NUMBER',
 'VICTIM_ROLE',
 'VICTIM_SEX',
 'VICTIM_AGE',
 'VICTIM_DEGREE_OF_INJURY',
 'VICTIM_SEATING_POSITION',
 'VICTIM_SAFETY_EQUIP1',
 'VICTIM_SAFETY_EQUIP2',
 'VICTIM_EJECTED']

In [5]:
victim_df.printSchema()

root
 |-- CASE_ID: string (nullable = true)
 |-- PARTY_NUMBER: integer (nullable = true)
 |-- VICTIM_ROLE: string (nullable = true)
 |-- VICTIM_SEX: string (nullable = true)
 |-- VICTIM_AGE: integer (nullable = true)
 |-- VICTIM_DEGREE_OF_INJURY: string (nullable = true)
 |-- VICTIM_SEATING_POSITION: string (nullable = true)
 |-- VICTIM_SAFETY_EQUIP1: string (nullable = true)
 |-- VICTIM_SAFETY_EQUIP2: string (nullable = true)
 |-- VICTIM_EJECTED: string (nullable = true)



In [6]:
victim_df.describe().show()

+-------+--------------------+------------------+------------------+------------------+------------------+-----------------------+-----------------------+--------------------+--------------------+------------------+
|summary|             CASE_ID|      PARTY_NUMBER|       VICTIM_ROLE|        VICTIM_SEX|        VICTIM_AGE|VICTIM_DEGREE_OF_INJURY|VICTIM_SEATING_POSITION|VICTIM_SAFETY_EQUIP1|VICTIM_SAFETY_EQUIP2|    VICTIM_EJECTED|
+-------+--------------------+------------------+------------------+------------------+------------------+-----------------------+-----------------------+--------------------+--------------------+------------------+
|  count|            30396065|          30396065|          30396065|          30396065|          30396065|               30396065|               30396065|            30396065|            28568078|          30396065|
|   mean|3.570956518533931...|1.6305697464457982|1.8703510818017473|3.5555555555555554| 63.75028152492765|     1.9722526757425833|      

In [7]:
victim_df.filter(victim_df.VICTIM_AGE == 0).count()

217309

In [8]:

# filter out unknown ages (998)
victim_df = victim_df.filter(victim_df.VICTIM_AGE != 998)

# change all 999 age instances to 0
victim_df = victim_df.withColumn('VICTIM_AGE', F.when(victim_df.VICTIM_AGE == 999, 0).otherwise(victim_df.VICTIM_AGE))


victim_df.describe().show()








+-------+--------------------+------------------+------------------+------------------+------------------+-----------------------+-----------------------+--------------------+--------------------+-------------------+
|summary|             CASE_ID|      PARTY_NUMBER|       VICTIM_ROLE|        VICTIM_SEX|        VICTIM_AGE|VICTIM_DEGREE_OF_INJURY|VICTIM_SEATING_POSITION|VICTIM_SAFETY_EQUIP1|VICTIM_SAFETY_EQUIP2|     VICTIM_EJECTED|
+-------+--------------------+------------------+------------------+------------------+------------------+-----------------------+-----------------------+--------------------+--------------------+-------------------+
|  count|            29360479|          29360479|          29360479|          29360479|          29360479|               29360479|               29360479|            29360479|            27573502|           29360479|
|   mean|3.624734272311125E17|1.6304027260590674|1.8611631661828245|3.5619047619047617|30.780437471745607|      2.009398080677406|  

In [9]:
victim_df.filter(victim_df.VICTIM_AGE == 0).count()

217824

In [10]:
# remove all "not stated - values" and null
filterCondition = (col(victim_df.columns[0]) != "-")
filterCondition = filterCondition | (col(victim_df.columns[0]) != None)


for c in victim_df.columns[1:]:
    filterCondition = filterCondition | (col(c) != "-")
    filterCondition = filterCondition | (col(c) != None)

filtered_df = victim_df.filter(filterCondition)
filtered_df.head(5)

[Row(CASE_ID=' 097293', PARTY_NUMBER=1, VICTIM_ROLE='2', VICTIM_SEX='M', VICTIM_AGE=20, VICTIM_DEGREE_OF_INJURY='0', VICTIM_SEATING_POSITION='3', VICTIM_SAFETY_EQUIP1='G', VICTIM_SAFETY_EQUIP2='-', VICTIM_EJECTED='0'),
 Row(CASE_ID=' 965874', PARTY_NUMBER=2, VICTIM_ROLE='2', VICTIM_SEX='M', VICTIM_AGE=19, VICTIM_DEGREE_OF_INJURY='0', VICTIM_SEATING_POSITION='3', VICTIM_SAFETY_EQUIP1='M', VICTIM_SAFETY_EQUIP2='G', VICTIM_EJECTED='0'),
 Row(CASE_ID='0000003', PARTY_NUMBER=1, VICTIM_ROLE='2', VICTIM_SEX='M', VICTIM_AGE=21, VICTIM_DEGREE_OF_INJURY='0', VICTIM_SEATING_POSITION='3', VICTIM_SAFETY_EQUIP1='G', VICTIM_SAFETY_EQUIP2='-', VICTIM_EJECTED='0'),
 Row(CASE_ID='0000005', PARTY_NUMBER=1, VICTIM_ROLE='1', VICTIM_SEX='M', VICTIM_AGE=44, VICTIM_DEGREE_OF_INJURY='2', VICTIM_SEATING_POSITION='1', VICTIM_SAFETY_EQUIP1='G', VICTIM_SAFETY_EQUIP2='-', VICTIM_EJECTED='0'),
 Row(CASE_ID='0000008', PARTY_NUMBER=1, VICTIM_ROLE='2', VICTIM_SEX='F', VICTIM_AGE=59, VICTIM_DEGREE_OF_INJURY='0', VICTIM_

In [13]:

encoded_and_index = ["VICTIM_SEX","VICTIM_DEGREE_OF_INJURY","VICTIM_SEATING_POSITION","VICTIM_SAFETY_EQUIP2","VICTIM_EJECTED"]

indexers = [StringIndexer(inputCol=c, outputCol=c+"_index") for c in encoded_and_index]
encoders = [OneHotEncoder(inputCol=c+"_index", outputCol=c+"_vec") for c in encoded_and_index]

encoded_df = filtered_df
for col_name in encoded_and_index:
    
    indexer = StringIndexer(inputCol=col_name, outputCol=col_name + "_index")
    print(indexer)
    encoded_df = indexer.fit(encoded_df).transform(encoded_df)
   
    encoder = OneHotEncoder(inputCol=col_name + "_index", outputCol=col_name + "_vec")
    encoded_df = encoder.fit(encoded_df).transform(encoded_df)

encoded_df.show(truncate=False)
encoded_df.columns

StringIndexer_6cf3ac687634
StringIndexer_d7253dc6fde2
StringIndexer_2bdd4c7948b7
StringIndexer_8bc06d6915c1
StringIndexer_4c7fe1665603
+-------+------------+-----------+----------+----------+-----------------------+-----------------------+--------------------+--------------------+--------------+----------------+--------------+-----------------------------+---------------------------+-----------------------------+---------------------------+--------------------------+------------------------+--------------------+------------------+
|CASE_ID|PARTY_NUMBER|VICTIM_ROLE|VICTIM_SEX|VICTIM_AGE|VICTIM_DEGREE_OF_INJURY|VICTIM_SEATING_POSITION|VICTIM_SAFETY_EQUIP1|VICTIM_SAFETY_EQUIP2|VICTIM_EJECTED|VICTIM_SEX_index|VICTIM_SEX_vec|VICTIM_DEGREE_OF_INJURY_index|VICTIM_DEGREE_OF_INJURY_vec|VICTIM_SEATING_POSITION_index|VICTIM_SEATING_POSITION_vec|VICTIM_SAFETY_EQUIP2_index|VICTIM_SAFETY_EQUIP2_vec|VICTIM_EJECTED_index|VICTIM_EJECTED_vec|
+-------+------------+-----------+----------+----------+-----

['CASE_ID',
 'PARTY_NUMBER',
 'VICTIM_ROLE',
 'VICTIM_SEX',
 'VICTIM_AGE',
 'VICTIM_DEGREE_OF_INJURY',
 'VICTIM_SEATING_POSITION',
 'VICTIM_SAFETY_EQUIP1',
 'VICTIM_SAFETY_EQUIP2',
 'VICTIM_EJECTED',
 'VICTIM_SEX_index',
 'VICTIM_SEX_vec',
 'VICTIM_DEGREE_OF_INJURY_index',
 'VICTIM_DEGREE_OF_INJURY_vec',
 'VICTIM_SEATING_POSITION_index',
 'VICTIM_SEATING_POSITION_vec',
 'VICTIM_SAFETY_EQUIP2_index',
 'VICTIM_SAFETY_EQUIP2_vec',
 'VICTIM_EJECTED_index',
 'VICTIM_EJECTED_vec']